## 基于同态加密实现和改进

In [7]:
# S:密钥 用于解密
# M:公钥 用于加密
# c:密文
# x:明文
# w:用于给明文x加权
# e:随机噪声

In [ ]:
#x ->加密->c
#x+x->加密->c+c
#x*10->加密->c*10

In [ ]:
# 神经网络中的加密

y  = relu(wx+b)

y_c = relu(w_c*x_c+b_c)

# 神经网络 分类 预测房价。。。

In [8]:
# 特别重要的公式 Sc = wx+e
# c = (wx+e)/S
# x = (Sc-e)/w
# 同态加密的关键步骤：公私钥对的生成，加密，解密
# 此外，同态加密还涉及数学运算

In [3]:
import numpy as np

### 首先实现一个简单的同态加密的例子

In [84]:
# 这一步中我们产生密钥S
def generate_key(w,m,n):
    S = (np.random.rand(m,n) * w / (2 ** 16)) 
    return S

In [85]:
# 这一步中对明文x进行加密，加密密钥为S，返回密文c
def encrypt(x,S,m,n,w):
    e = (np.random.rand(m))  # noise
    # # Sc = (wx+e)
    # (S^(-1))Sc =(S^(-1)) (wx+e)
    # c = (S^(-1)) (wx+e)
    c = np.linalg.inv(S).dot((w * x) + e) # 
    return c

In [86]:
# 这一步基于密文c,密钥S和权重w, 对密文进行解密
# x = (Sc)/w
# c——>x
def decrypt(c,S,w):
    x_recon = (S.dot(c) / w).astype('int')
    return x_recon

In [87]:
x = np.array([0,1,2,5]) # 明文
m = len(x) # 明文的长度 4
n = m
w = 16# 权重w
S = generate_key(w,m,n)
S # 产生密钥

array([[1.60535102e-04, 1.79058944e-05, 1.34292958e-05, 7.89049839e-05],
       [1.44160597e-04, 2.08471330e-04, 7.00835999e-05, 4.22527409e-05],
       [3.27200210e-05, 2.42835407e-04, 4.38227220e-05, 7.75260798e-05],
       [1.38743019e-04, 2.28236682e-06, 2.19884917e-04, 2.38584334e-04]])

### 下面探究加密算法的同态性

In [88]:
# 基于密钥对明文进行加密，得到密文c
x = np.array([0,1,2,5])
c = encrypt(x,S,m,n,w)
c

array([-119991.34741798,   44374.56128831,  221715.59407337,
        203379.64187519])

In [ ]:
# c+c解密之后得到x+x

In [89]:
# 基于权重w和密钥S对密文c进行解密
# c S w
x_recon = decrypt(c,S,w)
x_recon # 

array([0, 1, 2, 5])

In [90]:
# 1.对加法的同态性
decrypt(c+c+c,S,w)

array([ 0,  3,  6, 15])

In [91]:
# c+c解密后等于x+x

In [92]:
# 2.对乘法的同态性
# c*100解密后等于x*100
decrypt(c*100,S,w) 

array([  3, 100, 202, 504])

## 同态加密的改进算法

基于这篇paper来写的

https://eprint.iacr.org/2012/144.pdf

In [2]:
import numpy as np
from numpy.polynomial import polynomial as poly

def polymul(x, y, modulus, poly_mod):
    """Add two polynoms
    Args:
        x, y: two polynoms to be added.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z_modulus[X]/(poly_mod).
    """
    return np.int64(
        np.round(poly.polydiv(poly.polymul(x, y) % modulus, poly_mod)[1] % modulus)
    )


def polyadd(x, y, modulus, poly_mod):
    """Multiply two polynoms
    Args:
        x, y: two polynoms to be multiplied.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z_modulus[X]/(poly_mod).
    """
    return np.int64(
        np.round(poly.polydiv(poly.polyadd(x, y) % modulus, poly_mod)[1] % modulus)
    )

In [3]:
def gen_binary_poly(size):
    """Generates a polynomial with coeffecients in [0, 1]
    Args:
        size: number of coeffcients, size-1 being the degree of the
            polynomial.
    Returns:
        array of coefficients with the coeff[i] being 
        the coeff of x ^ i.
    """
    return np.random.randint(0, 2, size, dtype=np.int64)


def gen_uniform_poly(size, modulus):
    """Generates a polynomial with coeffecients being integers in Z_modulus
    Args:
        size: number of coeffcients, size-1 being the degree of the
            polynomial.
    Returns:
        array of coefficients with the coeff[i] being 
        the coeff of x ^ i.
    """
    return np.random.randint(0, modulus, size, dtype=np.int64)


def gen_normal_poly(size):
    """Generates a polynomial with coeffecients in a normal distribution
    of mean 0 and a standard deviation of 2, then discretize it.
    Args:
        size: number of coeffcients, size-1 being the degree of the
            polynomial.
    Returns:
        array of coefficients with the coeff[i] being 
        the coeff of x ^ i.
    """
    return np.int64(np.random.normal(0, 2, size=size))

In [4]:
def keygen(size, modulus, poly_mod):
    """Generate a public and secret keys
    Args:
        size: size of the polynoms for the public and secret keys.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        Public and secret key.
    """
    sk = gen_binary_poly(size)
    a = gen_uniform_poly(size, modulus)
    e = gen_normal_poly(size)
    b = polyadd(polymul(-a, sk, modulus, poly_mod), -e, modulus, poly_mod)
    return (b, a), sk

In [5]:
def encrypt(pk, size, q, t, poly_mod, pt):
    """Encrypt an integer.
    Args:
        pk: public-key.
        size: size of polynomials.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
        pt: integer to be encrypted.
    Returns:
        Tuple representing a ciphertext.      
    """
    # encode the integer into a plaintext polynomial
    m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    delta = q // t
    scaled_m = delta * m  % q
    e1 = gen_normal_poly(size)
    e2 = gen_normal_poly(size)
    u = gen_binary_poly(size)
    ct0 = polyadd(
            polyadd(
                polymul(pk[0], u, q, poly_mod),
                e1, q, poly_mod),
            scaled_m, q, poly_mod
        )
    ct1 = polyadd(
            polymul(pk[1], u, q, poly_mod),
            e2, q, poly_mod
        )
    return (ct0, ct1)

In [6]:
def decrypt(sk, size, q, t, poly_mod, ct):
    """Decrypt a ciphertext
    Args:
        sk: secret-key.
        size: size of polynomials.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
        ct: ciphertext.
    Returns:
        Integer representing the plaintext.
    """
    scaled_pt = polyadd(
            polymul(ct[1], sk, q, poly_mod),
            ct[0], q, poly_mod
        )
    decrypted_poly = np.round(scaled_pt * t / q) % t
    return int(decrypted_poly[0])

In [7]:
def add_plain(ct, pt, q, t, poly_mod):
    """Add a ciphertext and a plaintext.
    Args:
        ct: ciphertext.
        pt: integer to add.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
    Returns:
        Tuple representing a ciphertext.
    """
    size = len(poly_mod) - 1
    # encode the integer into a plaintext polynomial
    m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    delta = q // t
    scaled_m = delta * m  % q
    new_ct0 = polyadd(ct[0], scaled_m, q, poly_mod)
    return (new_ct0, ct[1])

In [8]:
def mul_plain(ct, pt, q, t, poly_mod):
    """Multiply a ciphertext and a plaintext.
    Args:
        ct: ciphertext.
        pt: integer to multiply.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
    Returns:
        Tuple representing a ciphertext.
    """
    size = len(poly_mod) - 1
    # encode the integer into a plaintext polynomial
    m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    new_c0 = polymul(ct[0], m, q, poly_mod)
    new_c1 = polymul(ct[1], m, q, poly_mod)
    return (new_c0, new_c1)

In [9]:
# Scheme's parameters
# polynomial modulus degree
n = 2**4
# ciphertext modulus
q = 2**15
# plaintext modulus
t = 2**8
# polynomial modulus
poly_mod = np.array([1] + [0] * (n - 1) + [1])
# Keygen
pk, sk = keygen(n, q, poly_mod)
# Encryption
pt1, pt2 = 73, 20
cst1, cst2 = 7, 5
ct1 = encrypt(pk, n, q, t, poly_mod, pt1)
ct2 = encrypt(pk, n, q, t, poly_mod, pt2)

print("[+] Ciphertext ct1({}):".format(pt1))
print("")
print("\t ct1_0:", ct1[0])
print("\t ct1_1:", ct1[1])
print("")
print("[+] Ciphertext ct2({}):".format(pt2))
print("")
print("\t ct1_0:", ct2[0])
print("\t ct1_1:", ct2[1])
print("")

# Evaluation
ct3 = add_plain(ct1, cst1, q, t, poly_mod)
ct4 = mul_plain(ct2, cst2, q, t, poly_mod)

# Decryption
decrypted_ct3 = decrypt(sk, n, q, t, poly_mod, ct3)
decrypted_ct4 = decrypt(sk, n, q, t, poly_mod, ct4)

print("[+] Decrypted ct3(ct1 + {}): {}".format(cst1, decrypted_ct3))
print("[+] Decrypted ct4(ct2 * {}): {}".format(cst2, decrypted_ct4))

[+] Ciphertext ct1(73):

	 ct1_0: [ 6116 29848  7278 18448  7380 22337  5846  1121 28389 10632 10282 22430
  4374 14996 23125 12470]
	 ct1_1: [30462 31772  6498  1289 26077 31630 30685 26996 28959 28495 23370  3159
 27057   257 32744 10700]

[+] Ciphertext ct2(20):

	 ct1_0: [32179 22194  7008 32205 29871 11280  6736 16484 15103  9473 19504 19186
 15938 21280 14353 17307]
	 ct1_1: [32080 28382 10342 11195 11104 15078 26834 29052 17580 14992  9316 23793
 22120 12313 28610 29243]

[+] Decrypted ct3(ct1 + 7): 80
[+] Decrypted ct4(ct2 * 5): 100


### 基于同态加密的简单的神经网络的实现基础：对

####  原理： 神经网络虽然能实现很强大的分类、预测等任务，但是其底层仍然是加减乘除和指数。
#### 因此可以用同态加密算法实现对简单的神经网络的加密

In [93]:
np.random.seed(1234)
input_dataset = np.array([[0.5],[0],[1],[0.2]])
output_dataset = np.array([[0],[1],[1],[0]])

In [99]:
x = np.array([5]) # 明文
m = len(x) # 明文的长度
n = m
e = (np.random.rand(m)) 
w = 16# 权重w
S = generate_key(w,m,n)
S # 产生密钥

array([[6.7496156e-05]])

In [102]:
def encrypt_vector_and_matrix(input_data):
    output_list = []
    m,n = input_data.shape
    # 输入必须是一个2维数组
    for i in range(m):
        for j in range(n):
            #print([input_dataset[i,j]])
            #print(S)
            #x = input_dataset[i,j]
            #c = np.linalg.inv(S).dot((w * x) + e)
            temp = encrypt(input_dataset[i,j],S,m,n,w)
            output_list.append(temp)
    output = np.array(output_list)
    output = output.reshape(input_data.shape)
    return output

In [105]:
# 加密输入数据和输出数据
input_dataset_encrypt = encrypt_vector_and_matrix(input_dataset)
#output_dataset_encrypt = encrypt_vector_and_matrix(output_dataset)

In [59]:
# 神经网络的结构如下：
input_dim = 3
hidden_dim = 4
output_dim = 1

# 基于神经网络的结构去生成权重
syn0_t = (np.random.randn(input_dim,hidden_dim) * 0.2) - 0.1
syn1_t = (np.random.randn(output_dim,hidden_dim) * 0.2) - 0.1

In [60]:
# 单向加密权重
syn0_encrypt = list()
for row in syn0_t:
    syn0_encrypt.append(encrypt(row,S,m,n,w).astype('int64'))
syn0_encrypt = np.array(syn0)

syn1_encrypt = list()
for row in syn1_t:
    syn1_encrypt.append(encrypt(row,S,m,n,w).astype('int64'))
syn1_encrypt = np.array(syn1)


In [61]:
# 定义激活函数
def sigmoid(before_activation):
    after_activation = 1/(1+np.exp(-before_activation))
    return after_activation

In [62]:
# 进行加密后的神经网络的前向传播
# 首先数据从输入层传输到隐藏层
input_dataset_encrypt.shape,syn0_encrypt.shape

((4, 4), (3, 4))

In [63]:
decrypt(syn0_encrypt,S,w)

ValueError: shapes (4,4) and (3,4) not aligned: 4 (dim 1) != 3 (dim 0)